In [13]:
# Imports
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import yfinance as yf
import requests
import hvplot.pandas
import json


In [14]:
# Import CSV
gdp_csv = pd.read_csv(Path("Resources/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3840536.csv"))
gdp_csv.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.549721e+09,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.931311e+10,1.972349e+10,2.149392e+10,2.573321e+10,2.352744e+10,2.681057e+10,...,9.433780e+11,9.505210e+11,9.642420e+11,9.848070e+11,9.199300e+11,8.733550e+11,9.853560e+11,1.012850e+12,1.009910e+12,9.207920e+11
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.780511e+10,1.990732e+10,2.014640e+10,2.049713e+10,1.913421e+10,1.811656e+10,1.875347e+10,1.805323e+10,1.879945e+10,2.011614e+10
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040428e+10,1.112805e+10,1.194335e+10,1.267652e+10,1.383858e+10,1.486247e+10,...,6.709630e+11,7.275710e+11,8.207880e+11,8.649670e+11,7.607300e+11,6.905430e+11,6.837420e+11,7.416920e+11,7.945720e+11,7.845880e+11
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.013530e+11,8.941719e+10,5.837598e+10


In [15]:
#S&P500 API call, GET JSON
url_chart = "https://api.twelvedata.com/time_series?apikey=18078c28ae5b492a8fc6f968053e43e7&interval=1month&format=JSON&type=index&exchange=NYSE&start_date=2000-01-01 06:37:00&end_date=2022-04-03 06:37:00&symbol=GSPC"
response_data_chart = requests.get(url=url_chart)
data_chart = response_data_chart.json()
print(json.dumps(data_chart, indent=4))

{
    "code": 400,
    "message": "write tcp 172.18.0.3:56608->172.104.131.97:5432: write: connection reset by peer",
    "status": "error",
    "meta": {
        "symbol": "GSPC",
        "interval": "1month",
        "exchange": "NYSE"
    }
}


In [3]:
load_dotenv()
yahoo_api = os.getenv("YAHOO_API_KEY")
type(yahoo_api)

str

In [22]:
gold = yf.download('GC=F', period='max')
gold_data = pd.DataFrame(gold['Close']).pct_change().dropna().rename(columns={"Close": "Gold"})


oil = yf.download("CL=F", period='max')
oil_data = pd.DataFrame(oil['Close']).pct_change().dropna().rename(columns={"Close": "Oil"})

snp = yf.download('^GSPC', period='max')
snp_data = pd.DataFrame(snp['Close']).pct_change().dropna().rename(columns={"Close": "S&P 500"})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Gold
Date,
2000-08-31,0.016064
2000-09-01,-0.004671
2000-09-05,-0.004332
2000-09-06,-0.005801
2000-09-07,-0.000729


In [48]:
today = pd.to_datetime('today')
print(today)

2022-04-04 20:54:41.884605


In [47]:
# Slice out Data
# GFC

gold_data_gfc = gold_data.loc["2006-06-01":"2010-01-01"]
oil_data_gfc = oil_data.loc["2006-06-01":"2010-01-01"]
snp_data_gfc = snp_data.loc["2006-06-01":"2010-01-01"]

# 9/11

gold_data_911 = gold_data.loc["2000-10-07":"2005-08-30"]
oil_data_911 = oil_data.loc["2000-10-07":"2005-08-30"]
snp_data_911 = snp_data.loc["2000-10-07":"2005-08-30"]

# COVID-19

gold_data_covid = gold_data.loc["2018-12-31":today]
oil_data_covid = oil_data.loc["2018-12-31":today]
snp_data_covid = snp_data.loc["2018-12-31":today]

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War

gold_data_long = gold_data.loc["2000-10-07":"2012-12-15"]
oil_data_long = oil_data.loc["2000-10-07":"2012-12-15"]
snp_data_long = snp_data.loc["2000-10-07":"2012-12-15"]

# What are the current short-term effects of war? (Ukraine Russia War)

gold_data_short = gold_data.loc["2022-02-24":today]
oil_data_short = oil_data.loc["2022-02-24":today]
snp_data_short = snp_data.loc["2022-02-24":today]


In [46]:
# Concat by every event
# GFC
event_gfc = pd.concat(
    [gold_data_gfc, oil_data_gfc, snp_data_gfc], axis="columns", join="inner"
)
# 9/11

event_911 = pd.concat(
    [gold_data_911, oil_data_911, snp_data_911], axis="columns", join="inner"
)

# COVID-19

event_covid = pd.concat(
    [gold_data_covid, oil_data_covid, snp_data_covid], axis="columns", join="inner"
)

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War

event_long = pd.concat(
    [gold_data_long, oil_data_long, snp_data_long], axis="columns", join="inner"
)

# What are the current short-term effects of war? (Ukraine Russia War)

event_short = pd.concat(
    [gold_data_short, oil_data_short, snp_data_short], axis="columns", join="inner"
)